In [1]:
#Import ALL dependencies

import pandas as pd
import seaborn as sns
import numpy as np
import csv

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from sqlalchemy import func
from flask import Flask, jsonify
import datetime as dt

from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

from sqlalchemy import Column, Integer, String, Float

import numpy as np

In [2]:
#read the two csv's in question
measurment_df = pd.read_csv("Resources//Data//measurement.csv")
station_df = pd.read_csv("Resources//Data//station.csv")

In [3]:
#Start to clean measurement csv
measurment_df.count()
measurment_df.dtypes

id           int64
station     object
date        object
prcp       float64
tobs         int64
dtype: object

In [4]:
measurment_df['prcp'].fillna(0, inplace=True)
#measurment_df.describe()
measurment_df.head()
del measurment_df['id']
measurment_df.head()

,station,date,prcp,tobs
0,USC00519397,1/1/2010,0.08,65
1,USC00519397,1/2/2010,0.00,63
2,USC00519397,1/3/2010,0.00,74
3,USC00519397,1/4/2010,0.00,76
4,USC00519397,1/6/2010,0.00,73


In [5]:
#save to csv
measurment_df.to_csv("Resources//Data//clean_measurement.csv", index=False, header=True)

In [6]:
station_df.head()

,id,station,name,latitude,longitude,elevation
0,1,USC00519397,"WAIKIKI 717.2, HI US",21.2716,-157.8168,3.0
1,2,USC00513117,"KANEOHE 838.1, HI US",21.4234,-157.8015,14.6
2,3,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.5213,-157.8374,7.0
3,4,USC00517948,"PEARL CITY, HI US",21.3934,-157.9751,11.9
4,5,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.4992,-158.0111,306.6


In [7]:
#Start to clean station 
station_df.head()
station_df.count()

#need to drop last column
del station_df['id']
station_df.head()

,station,name,latitude,longitude,elevation
0,USC00519397,"WAIKIKI 717.2, HI US",21.2716,-157.8168,3.0
1,USC00513117,"KANEOHE 838.1, HI US",21.4234,-157.8015,14.6
2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.5213,-157.8374,7.0
3,USC00517948,"PEARL CITY, HI US",21.3934,-157.9751,11.9
4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.4992,-158.0111,306.6


In [8]:
station_df.dtypes


station       object
name          object
latitude     float64
longitude    float64
elevation    float64
dtype: object

In [9]:
#save to a new csv
station_df.to_csv('Resources//Data//clean_station.csv', index=False, header=True)

In [10]:
station_df.head()

,station,name,latitude,longitude,elevation
0,USC00519397,"WAIKIKI 717.2, HI US",21.2716,-157.8168,3.0
1,USC00513117,"KANEOHE 838.1, HI US",21.4234,-157.8015,14.6
2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.5213,-157.8374,7.0
3,USC00517948,"PEARL CITY, HI US",21.3934,-157.9751,11.9
4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.4992,-158.0111,306.6


In [11]:
measurment_df.head()

,station,date,prcp,tobs
0,USC00519397,1/1/2010,0.08,65
1,USC00519397,1/2/2010,0.00,63
2,USC00519397,1/3/2010,0.00,74
3,USC00519397,1/4/2010,0.00,76
4,USC00519397,1/6/2010,0.00,73


In [12]:
measurment_df.head()

,station,date,prcp,tobs
0,USC00519397,1/1/2010,0.08,65
1,USC00519397,1/2/2010,0.00,63
2,USC00519397,1/3/2010,0.00,74
3,USC00519397,1/4/2010,0.00,76
4,USC00519397,1/6/2010,0.00,73


In [13]:
station_df

,station,name,latitude,longitude,elevation
0,USC00519397,"WAIKIKI 717.2, HI US",21.27160,-157.81680,3.0
1,USC00513117,"KANEOHE 838.1, HI US",21.42340,-157.80150,14.6
2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.52130,-157.83740,7.0
3,USC00517948,"PEARL CITY, HI US",21.39340,-157.97510,11.9
4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.49920,-158.01110,306.6
5,USC00519523,"WAIMANALO EXPERIMENTAL FARM, HI US",21.33556,-157.71140,19.5
6,USC00519281,"WAIHEE 837.5, HI US",21.45167,-157.84889,32.9
7,USC00511918,"HONOLULU OBSERVATORY 702.2, HI US",21.31520,-157.99920,0.9
8,USC00516128,"MANOA LYON ARBO 785.2, HI US",21.33310,-157.80250,152.4


In [14]:
#Create your ORM classes, first measurement:
class Measurment(Base):
    __tablename__ = 'measurement'
    __table_args__ = {'extend_existing': True} 
    id = Column(Integer, primary_key=True)
    station = Column(String(255))
    date = Column(String(255))
    prcp = Column(Float)
    tobs = Column(Integer)

In [15]:
class Station(Base):
    __tablename__ = 'station'
    id = Column(Integer, primary_key=True)
    __table_args__ = {'extend_existing': True} 
    station = Column(String(255))
    name = Column(String(255))
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)

In [16]:
Base.metadata.create_all(engine)

NameError: name 'engine' is not defined

In [ ]:
measurment_df.to_sql('measurement', engine, if_exists = "append", index=False)
station_df.to_sql('station', engine, if_exists = "append", index=False)